# Display sequencing emo-bon efforts across European sites
- At first showing the validated and relessed data from [emo-bon-data-validataion](https://github.com/emo-bon/emo-bon-data-validation/tree/main/validated-data)
- Second, ask Cymon what metadata can be shown about data which are not ready/released yet.
- I use `leafmap` for GIS integration

In [1]:
# system dependent setup
import sys
import os
import io
import logging
logger = logging.getLogger(name="Diversity analysis app")

if 'google.colab' in str(get_ipython()):
    # clone the momics-demos repository to use the utils module from there
    # TODO: eventually utils from momics will be used for that
    try:
        os.system('git clone https://github.com/palec87/momics-demos.git')
        logger.info(f"Repository cloned")
    except OSError as e:
        logger.info(f"An error occurred while cloning the repository: {e}")

    sys.path.insert(0,'/content/momics-demos')

else:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # local utils, to be removed in the future

    # downside of this is that all the deps need to be installed in the current (momics-demos) environment
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../marine-omics')))  # local momics package, to be removed too

from utils import init_setup, get_notebook_environment
init_setup()

# Initialize the environment variable
notebook_environment = 'unknown'
# Determine the notebook environment
env = get_notebook_environment()
logger.info(f"Environment: {env}")

Platform: local Linux


## Imports

In [2]:
import requests
import pandas as pd
# import leafmap.leafmap as leafmap
import leafmap

In [3]:
url_obs = "https://raw.githubusercontent.com/emo-bon/emo-bon-data-validation/refs/heads/main/validated-data/Observatory_combined_logsheets_validated.csv"
url_data = "https://raw.githubusercontent.com/emo-bon/emo-bon-data-validation/refs/heads/main/validated-data/Batch1and2_combined_logsheets_2024-11-12.csv"


df_obs = pd.read_csv(url_obs ,index_col=0)
df_data = pd.read_csv(url_data ,index_col=0)

df_obs.columns, df_data.columns

(Index(['project_name', 'latitude', 'longitude', 'geo_loc_name',
        'loc_broad_ocean', 'loc_broad_ocean_mrgid', 'loc_regional',
        'loc_regional_mrgid', 'loc_loc', 'loc_loc_mrgid', 'env_broad_biome',
        'env_local', 'env_package', 'tot_depth_water_col', 'organization',
        'organization_country', 'organization_edmoid', 'wa_id',
        'extra_site_info', 'contact_name', 'contact_email', 'contact_orcid',
        'ENA_accession_number_umbrella', 'ENA_accession_number_project'],
       dtype='object'),
 Index(['source_mat_id_orig', 'samp_description', 'tax_id', 'scientific_name',
        'investigation_type', 'env_material', 'collection_date',
        'sampling_event', 'sampl_person', 'sampl_person_orcid',
        ...
        'silicate_method', 'sulfate', 'sulfate_method', 'sulfide',
        'sulfide_method', 'turbidity', 'turbidity_method', 'water_current',
        'water_current_method', 'env_package'],
       dtype='object', length=113))

In [22]:
m = leafmap.Map(center=(50, 10), zoom=4)
m.add_points_from_xy(
    df_obs, x="longitude", y="latitude",
    popup=['organization', "contact_name", "contact_email", "ENA_accession_number_umbrella", 'tot_depth_water_col'],
    layer_name="EMO-BON Observatories")
m

Map(center=[50, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [62]:
url2 = "https://docs.google.com/spreadsheets/d/1j9tRRsRCcyViDMTB1X7lx8POY1P5bV7UijxKKSebZAM/edit?gid=0&format=xslx"
# df_data = pd.read_csv(url2, delimiter=";", on_bad_lines="warn", skiprows=100)
df_data = pd.read_excel(url2, skiprows=100)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [59]:
df_data.head()

,,,,,,,,,,,,,,,,,,try{
var Oh=function(a){_.U.call(this),this.B=a,this.v=null,this.o={},this.C={},this.i={},this.j=null},"_.A(Oh,_.U)",_.Ph=function(a){if(a.v)return a.v,for(var b in a.i)if(a.i[b].ve()&&a.i[b].eb())return a.i[b],return null},Oh.prototype.A=function(a){this.i[a]&&(_.Ph(this)&&_.Ph(this).yc()==a||this.i[a].qd(!0))},Oh.prototype.Pa=function(a){this.j=a,for(var b in this.i)this.i[b].ve()&&this.i[b].Pa(a)},"_.Qh=function(a,b){a.i[b.yc()]=b}",Oh.prototype.Qb=function(a){return a in this.i?this.i[a]:null},var Rh=new Oh(_.Md),"_.Qd(""dd"",Rh)",NaN
}catch(e){_._DumpException(e)},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"_.hj=function(a,b){return _.P(a,36,b)}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
}catch(e){_._DumpException(e)},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
from io import BytesIO

r = requests.get('https://docs.google.com/spreadsheets/d/1j9tRRsRCcyViDMTB1X7lx8POY1P5bV7UijxKKSebZAM/edit?gid=0&format=csv')
data = r.content
df = pd.read_csv(BytesIO(data), index_col=0)


ParserError: Error tokenizing data. C error: Expected 669 fields in line 80, saw 4658


#